In [1]:
using Revise

using RigidBodyDynamics
using RigidBodyDynamics: Bounds

using RigidBodyTreeInspector
using DrakeVisualizer
using Plots

using BilevelTrajOpt

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9dd84cf97c4fc16956f39829c81f3b6907c8fc83-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9dd84cf97c4fc16956f39829c81f3b6907c8fc83-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [2]:
urdf = joinpath("..", "urdf", "ball.urdf")
mechanism = parse_urdf(Float64, urdf)
body = findbody(mechanism, "ball")
basejoint = joint_to_parent(body, mechanism)
floatingjoint = Joint(basejoint.name, frame_before(basejoint), frame_after(basejoint), QuaternionFloating{Float64}())
replace_joint!(mechanism, basejoint, floatingjoint)
position_bounds(floatingjoint) .= Bounds(-100, 100)
velocity_bounds(floatingjoint) .= Bounds(-100, 100)
env = parse_contacts(mechanism, urdf, .5);
x0 = MechanismState(mechanism)
Δt = 0.01
sim_data = get_sim_data(x0,env,Δt,true);

In [27]:
# q0 = [1., 0., 0., 0., 0., 0., 0.]
# v0 = [0., 0., 0., .25, 0., 0.]
# u0 = zeros(sim_data.num_v)
# set_configuration!(x0, q0)
# set_velocity!(x0, v0)
# setdirty!(x0)
# traj = BilevelTrajOpt.simulate(x0,env,sim_data.Δt,1,implicit_contact=false)
# qnext = traj[1:sim_data.num_q,2]
# vnext = traj[sim_data.num_q+1:sim_data.num_q+sim_data.num_v,2]

q0 = [1.0,7.14e-12,7.14e-12,7.14e-12,0.016266,7.52931e-12,-3.01284e-9]
v0 = [7.0e-12,7.0e-12,7.0e-12,0.126601,6.52901e-12,-0.439901]
u0 = zeros(sim_data.num_v)
qnext = [1.0,8.18e-12,8.18e-12,8.18e-12,0.016266,8.52931e-12,-4.08867e-9 ]
vnext = [ 8.0e-12,8.0e-12,8.0e-12,9.05748e-9,1.42951e-18,-1.07683e-7 ]

z0 = zeros(sim_data.num_contacts*(sim_data.β_dim+2))

num_steps_default = 10
α_vect_default = [1.^i for i in 1:num_steps_default]
c_vect_default = [50.+1.^i for i in 1:num_steps_default]
I_vect_default = 1e-6*ones(num_steps_default)

τ_ip, x_sol_ip = solve_implicit_contact_τ(sim_data,q0,v0,u0,z0,qnext,vnext,ip_method=true);
τ_auglag, x_sol_auglag = solve_implicit_contact_τ(sim_data,q0,v0,u0,z0,qnext,vnext,ip_method=false,α_vect=α_vect_default,c_vect=c_vect_default,I_vect=I_vect_default);

6-element Array{Float64,1}:
 -2.24231e-8 
  1.47961e-9 
 -3.43975e-10
 -1.47303e-8 
 -3.43975e-10
 -3.37529e-10

Solve_Succeeded


In [51]:
display("IP")
display(x_sol_ip)
display("Aug Lag")
display(x_sol_auglag)
display("Explicit")
display([0.0,0.302637,0.214285,1.59316,0.214285,8.04123e-9,5.48421])
# display(traj[sim_data.num_q+sim_data.num_v+1:end,2])

6-element Array{Float64,1}:
  0.633397   
  7.90339e-11
 -0.633397   
 -7.9034e-11 
 -4.61931e-7 
 -2.69167    

6-element Array{Float64,1}:
  0.508119   
  0.000223622
 -1.24265    
  4.30394e-6 
 -0.081369   
 -5.33468    

6-element Array{Float64,1}:
 1396.66      
 1397.29      
 1397.91      
 1397.29      
    0.00422715
 -695.976     

6-element Array{Float64,1}:
    -5.59133e-5
 -1673.26      
    -8.14415e-5
 -1673.27      
     0.0175127 
     2.86739e-5

6-element Array{Float64,1}:
 1324.01      
 1948.83      
 1455.65      
 1948.83      
   -0.00418368
 -974.416     

...

7-element Array{Float64,1}:
 0.0       
 0.302637  
 0.214285  
 1.59316   
 0.214285  
 8.04123e-9
 5.48421   

"IP"

6-element Array{Float64,1}:
  0.182353   
  0.364566   
  1.47288    
  0.364566   
 -9.43518e-10
  5.48421    

"Aug Lag"

6-element Array{Float64,1}:
 -6.17847   
 12.043     
 -4.87007   
 12.0252    
 -1.20713e-5
  5.4575    

In [ ]:
ForwardDiff.jacobian(x̃ -> solve_implicit_contact_τ(sim_data,q0,v0,u0,z0,x̃[1:sim_data.num_q],x̃[sim_data.num_q+1:sim_data.num_q+sim_data.num_v];ip_method=false)[1],vcat(qnext,vnext))